# Исследование надежности заемщиков.

### Шаг 1. Обзор данных

In [2]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv') # чтение содержимого файла и сохранение в датафрейме data
print(data.head()) # знакомство с данными
data.info() # оценка объема данных в таблице

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0      M   сотрудник     0  145885.952297   
3   женат / замужем                 0      M   сотрудник     0  267628.550329   
4  гражданский брак                 1      F   пенсионер     0  158616.077870   

                      purpose  
0               покупка жилья  
1     приобретение автомобиля  
2               покупка жи

### Шаг 2.1 Заполнение пропусков

In [3]:
# Из полученного обзора данных выявлено, что в столбцах 'days_employed' и 'total_income' имеются пропущенные значения
# Определим долю пропущенных значений в столбце 'days_employed'
print(data['days_employed'].count() / data['children'].count())
# также определим долю пропущенных в столбце 'total_income'
print(data['total_income'].count()/data['children'].count())

# Исходя из одинакового количества пропусков с столбцах 'days_employed' и 'total_income' можно предположить, 
# что возможная причина пропусков, это отсутствие трудового стажа заемщика, и соответственно, отсутствие ежемесячного дохода.

# Медианное значение корректнее применять, когда некоторые значения сильно выделяются среди остального большинства.
# заполнение пропусков медианым значением столбца 'days_employed'
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())
# заполнение пропусков медианым значением столбца 'total_income'
data['total_income'] = data['total_income'].fillna(data['total_income'].median())

# проверка, пропуски отсутствуют, тип столбцов 'days_employed' и 'total_income' остался прежним 'float64'
data.info()

0.8990011614401858
0.8990011614401858
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Шаг 2.2 Проверка данных на аномалии и исправления.

In [4]:
# В целях исключения аномалий обратим внимание на столбец 'days_employed', в нем имеются отрицательные значения,
# которые соответствуют возможному стажу заемщика при соответствии количества прожитых лет.
# В целях исключения аномалий в столбце days_employed', изменим данные с отрицательного на положительные.
data ['days_employed'] = data.loc[:,'days_employed'].abs()
print(data['days_employed'].head(5))

0      8437.673028
1      4024.803754
2      5623.422610
3      4124.747207
4    340266.072047
Name: days_employed, dtype: float64


In [5]:
# также видно, что в данном столбце остались аномальные данные с величиной стажа, превышающем 100 лет (например, строка 5)
# рассчитаем количество строк с величиной стажа более 45 лет.
print (data['days_employed'][data['days_employed'] > 16000].count())

3450


In [6]:
# рассчитаем медианное значение по столбцу 'days_employed' при условии стажа менее 30 лет.
print(data[data['days_employed'] < 10950]['days_employed'].median()) # получилось 1328.18 дней
# заполним аномальные значения медианым значением столбца 'days_employed' при стаже менее 30 лет.
data.loc[data['days_employed'] > 10950, 'days_employed'] = '1328.18'

1328.1808865308271


In [7]:
# Кроме того, в столбце 'gender'обнаружился пол клиента с индексом 'XNA' в количестве 1 шт.
# удалим данную строку в связи с незначительной величиной.
data.loc[data['gender'] == 'XNA','gender'] = None
data = data.dropna(subset=['gender'])
# проверка столбца 'gender' после удаления, остались только F и M
print(data['gender'].value_counts())

F    14236
M     7288
Name: gender, dtype: int64


In [8]:
# в столбце 'children' обнаружилось количество детей равное "-1" в кол-ве 1 шт. и количество детей равное 20 - 76 шт.
# Появление данных аномальных величин можеет быт вызвано "человеческим" фактором при занесении данных в таблицу
# также удалим данные строки в связи с незначительной величиной.
data.loc[data['children'] == -1, 'children'] = None
data.loc[data['children'] == 20,'children'] = None
data = data.dropna(subset=['children'])
print(data['children'].value_counts())

0.0    14148
1.0     4818
2.0     2055
3.0      330
4.0       41
5.0        9
Name: children, dtype: int64


### Шаг 2.3. Изменение типов данных.

In [9]:
# Заменим вещественный тип данных в столбце 'total_income' на целочисленный.
data['total_income'] = data['total_income'].astype('int')
print(data['total_income'].head())

0    253875
1    112080
2    145885
3    267628
4    158616
Name: total_income, dtype: int32


### Шаг 2.4. Удаление дубликатов.

In [10]:
# для исключения не явных дубликатов в столбце 'education' все символы в столбце 'education' приведем к нижнему регистру
data['education'] = data['education'].str.lower()
print(data['education'].unique()) # выведем все уникальные значения столбца 'education'

# Для удаления дубликатов проведена обработка данных:
# пропуски в столбцах заполнены в пункте 2.1;
# аномальные значения обработаны в пункте 2.2;
# не явные дубли (приведение к единому регистру) были обработаны в предыдущем задании;
# осталось удалить явные дубли:
 
print(data.duplicated().sum()) # количество дубликатов - 71
# очистим таблицу от дубликатов с новой индексацией и удалением старой индексации
data = data.drop_duplicates().reset_index(drop=True)
# проверим количество дубликатов в таблице
print(data.duplicated().sum()) # количество дубликатов - 0

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']
71
0


### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [11]:
# Создаем два новых датафрейма:
# первый со столбцами 'education_id' и 'education'
data_education = data[['education_id','education']]
print(data_education.head(5))
# второй со столбцами 'family_status_id' и 'family_status'
data_family = data[['family_status_id','family_status']]
print(data_family.head(5))
# удалим из исходного датафрейма столбцы 'education' и 'family_status'
data.drop(['education','family_status'], inplace=True, axis=1)
print(data.head(5))

   education_id education
0             0    высшее
1             1   среднее
2             1   среднее
3             1   среднее
4             1   среднее
   family_status_id     family_status
0                 0   женат / замужем
1                 0   женат / замужем
2                 0   женат / замужем
3                 0   женат / замужем
4                 1  гражданский брак
   children days_employed  dob_years  education_id  family_status_id gender  \
0       1.0   8437.673028         42             0                 0      F   
1       1.0   4024.803754         36             1                 0      F   
2       0.0    5623.42261         33             1                 0      M   
3       3.0   4124.747207         32             1                 0      M   
4       0.0       1328.18         53             1                 1      F   

  income_type  debt  total_income                     purpose  
0   сотрудник     0        253875               покупка жилья  
1   сотрудник

### Шаг 2.6. Категоризация дохода.

In [12]:
# создаем функцию для указания заданных категорий в столбце "total_income_category"
def income_group(income):
    if income <= 30000:
        return 'E'
    if income <= 50000:
        return 'D'
    if income <= 200000:
        return 'C'
    if income <= 1000000:
        return 'B'
    return 'A'
# создаем столбец "total_income_category" с вышеуказанными категориями 
data['total_income_category'] = data['total_income'].apply(income_group)
display(data['total_income_category'].value_counts()) # выводим данные из столбца "total_income_category"

C    15922
B     5012
D      349
A       25
E       22
Name: total_income_category, dtype: int64

### Шаг 2.7. Категоризация целей кредита.

In [14]:
# создаем функцию, которая на основании данных из столбца "purpose" сформирует новый столбец "purpose_category"
# с данными по категориям
def purpose_categoried(purpose):
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'авто' in purpose:
        return 'операции с автомобилем'
    elif 'образован' in purpose:
        return 'получение образования'
    else:
# по категории 'операции с недвижимостью' функция будет присваивать данные при не выполнении всех условий        
        return 'операции с недвижимостью' 
data['purpose_category'] = data['purpose'].apply(purpose_categoried)
display(data['purpose_category'].value_counts())

операции с недвижимостью    10750
операции с автомобилем       4279
получение образования        3988
проведение свадьбы           2313
Name: purpose_category, dtype: int64

### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [15]:
# рассчитаем долю не возврата кредита в зависимости от количества детей:
display(data.pivot_table(index=['children'], values='debt', aggfunc=['sum','count','mean']))

,sum,count,mean
,debt,debt,debt
children,,,
0.0,1063,14090,0.075444
1.0,444,4808,0.092346
2.0,194,2052,0.094542
3.0,27,330,0.081818
4.0,4,41,0.097561
5.0,0,9,0.000000


**Ответ:**

При анализе данных выявлено, максимальный % не возврата кредита у клиентов с 4 детьми (хотя таких случаев всего 41). При количестве одного и двух детей процент невозврата практически одинаковый (9,23 и 9,45 соответственно), хотя количество случаев при 1 ребенке более чем в 2 раза превышает при 2 детях. У клиентов с 5 детьми отсутствуют случаи не возврата кредита в срок.

**Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [16]:
# рассчитаем процент не возврата кредита в зависимости от семейного положения:
# для этого заменим в столбце 'family_status_id' имеющиеся цифры на понятия в соответствии таблицей соответствия:
data['family_status_id'] = data['family_status_id'].replace({0:'женат / замужем', 1:'гражданский брак', 2:'Не женат / не замужем', 3:'в разводе',4:'вдовец / вдова'})
# рассчитаем долю не возврата кредита в зависимости от семейного положения:
display(data.pivot_table(index=['family_status_id'], values='debt', aggfunc=['sum','count','mean']))

,sum,count,mean
,debt,debt,debt
family_status_id,,,
Не женат / не замужем,63,951,0.066246
в разводе,84,1189,0.070648
вдовец / вдова,273,2796,0.097639
гражданский брак,385,4133,0.093153
женат / замужем,927,12261,0.075606


**Ответ:**

При анализе данных выявлено, максимальный % не возврата кредита у клиентов с семейным положением "вдовец / вдова" и "гражданский брак" (9,76 и 9,28 соответственно). Что вполне естественно с моей точки зрения, т.к. существует либо недостаток средств в случае "вдовы", либо отсутствие "семейного" бюджета" при проживании в "гражданском браке". У клиентов с семейным положением "не женат" самая низкий % не возврата - 6,62.

**Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [17]:
# рассчитаем долю не возврата кредита в зависимости от уровня дохода:
display(data.pivot_table(index=['total_income_category'], values='debt', aggfunc=['sum','count','mean']))

,sum,count,mean
,debt,debt,debt
total_income_category,,,
A,2,25,0.080000
B,354,5012,0.070630
C,1353,15922,0.084977
D,21,349,0.060172
E,2,22,0.090909


**Ответ:**

При анализе данных выявлено, максимальный % не возврата кредита у клиентов с уровнем дохода категории E (0–30000) - 9.09. У клиентов с уровнем дохода категории A (выше 1 000 001) и C (50 001–200 000) приблизительно одинаковый % не возврата (8 и 8,49 соответственно). Самый низкий % не возврата у клиентов с уровнем дохода категории D (30 001–50 000) - 6.17.

**Вопрос 4: Как разные цели кредита влияют на его возврат в срок?**

In [18]:
# рассчитаем долю не возврата кредита в зависимости от цели кредита:
display(data.pivot_table(index=['purpose_category'], values='debt', aggfunc=['sum','count','mean']))

,sum,count,mean
,debt,debt,debt
purpose_category,,,
операции с автомобилем,400,4279,0.093480
операции с недвижимостью,780,10750,0.072558
получение образования,369,3988,0.092528
проведение свадьбы,183,2313,0.079118


**Ответ:**

При анализе данных выявлено, максимальный % не возврата кредита у клиентов с целью кредита "операции с автомобилем" и "получение образования" (9,34 и 9,25 соответственно). Самый низкий % не возврата у клиентов с целью кредита "операции с недвижимостью"- 7,25. Количество клиентов с целью кредита "операции с недвижимостью" сопоставимо с суммой количества клиентов по оставшимся трем категориям.

## Общий вывод:

При проведении исследования предоставленой банком статистики о платёжеспособности клиентов были выявлены зависимости между семейным положением клиента и количеством детей на факт погашения кредита в срок. Максимальный процент не возврата кредита у клиентов с семейным положением "вдовец / вдова" и "гражданский брак" (9,76 и 9,28 соответственно), а также у клиентов с 4 детьми (9.76). Клиенты, входящие в данные категории являются клиентами с максимальной степенью риска не возврата кредита.
Надежнее выдавать кредиты клиентам с отсутствием детей и семейным положением "женат / замужем", данные категории с больше вероятностью смогут погасить кредиты в установленные договором сроки. Кроме того, клиенты из указанных категорий более часто обращаются в банк за предоставлением кредита, что также более выгодно для банка.